In [1]:
from joblib import dump, load

In [2]:
x_train = load('x_train_rdkit.pkl')
x_test = load('x_test_rdkit.pkl')
x_valid = load('x_valid_rdkit.pkl')

In [3]:
y_train = load('y_train.pkl')
y_test = load('y_test.pkl')
y_valid = load('y_valid.pkl')

In [4]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [5]:
seed = 42

In [6]:
cv = StratifiedKFold(n_splits=10, random_state=seed)

D:\Anaconda3\envs\deepchem\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
scale = StandardScaler().fit(x_train)
x_train_scaled = scale.transform(x_train)
x_test_scaled = scale.transform(x_test)
x_valid_scaled = scale.transform(x_valid)

In [9]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [48]:
param_grid = {'p': np.arange(1, 11, 1)}

In [49]:
knn = GridSearchCV(KNeighborsClassifier(n_neighbors=4, weights='distance'), param_grid=param_grid, cv=cv, n_jobs=-1, verbose=1)

In [50]:
knn.fit(x_train_scaled, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.6s finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=False),
             error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=4, p=2,
                                            weights='distance'),
             iid='deprecated', n_jobs=-1,
             param_grid={'p': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [51]:
knn.best_params_

{'p': 4}

In [52]:
knn.best_score_

0.8058823529411765

In [67]:
knn.cv_results_['mean_test_score']

array([0.79411765, 0.78529412, 0.79705882, 0.80588235, 0.79117647,
       0.78823529, 0.77058824, 0.77058824, 0.77647059, 0.77647059])

In [68]:
knn.cv_results_['params']

[{'p': 1},
 {'p': 2},
 {'p': 3},
 {'p': 4},
 {'p': 5},
 {'p': 6},
 {'p': 7},
 {'p': 8},
 {'p': 9},
 {'p': 10}]

In [154]:
knn_2c8 = KNeighborsClassifier(n_neighbors=4, weights='distance', p=1)

In [155]:
knn_2c8.fit(x_train_scaled, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=1,
                     weights='distance')

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import matthews_corrcoef, accuracy_score, roc_auc_score, make_scorer, recall_score

In [18]:
matthews = make_scorer(matthews_corrcoef)
specificity = make_scorer(recall_score, pos_label=0)

In [156]:
cross_val_score(knn_2c8, x_train_scaled, y_train, cv=cv, scoring='accuracy').mean()

0.7941176470588235

In [157]:
cross_val_score(knn_2c8, x_train_scaled, y_train, cv=cv, scoring='roc_auc').mean()

0.8446491228070176

In [158]:
cross_val_score(knn_2c8, x_train_scaled, y_train, cv=cv, scoring=matthews).mean()

0.5884729368445946

In [159]:
cross_val_score(knn_2c8, x_train_scaled, y_train, cv=cv, scoring='recall').mean()

0.7942105263157895

In [160]:
cross_val_score(knn_2c8, x_train_scaled, y_train, cv=cv, scoring=specificity).mean()

0.7952380952380952

In [161]:
y_pred = knn_2c8.predict(x_test_scaled)

In [162]:
y_pred

array([1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0])

In [163]:
accuracy_score(y_test, y_pred)

0.7850467289719626

In [164]:
roc_auc_score(y_test, knn_2c8.predict_proba(x_test_scaled)[:,1])

0.832039911308204

In [165]:
matthews_corrcoef(y_test, y_pred)

0.564609859025163

In [166]:
recall_score(y_test, y_pred)

0.7727272727272727

In [167]:
recall_score(y_test, y_pred, pos_label=0)

0.8048780487804879

In [83]:
import matplotlib.pyplot as plt

0.7941176470588235 4
0.822429906542056 5


In [151]:
training_acc = []
test_acc = []

for i in np.arange(1, 11, 1):
    knn_2c8 = KNeighborsClassifier(n_neighbors=i, weights='distance', p=1)
    knn_2c8.fit(x_train_scaled, y_train)
    training_acc.append(cross_val_score(knn_2c8, x_train_scaled, y_train, cv=cv, scoring='accuracy').mean())
    y_pred = knn_2c8.predict(x_test_scaled)
    test_acc.append(accuracy_score(y_test, y_pred))
print(max(training_acc),(training_acc.index(max(training_acc)))+1) # index starts from 0
print(max(test_acc),(test_acc.index(max(test_acc)))+1)
plt.figure(figsize=(30, 10))
plt.plot(np.arange(1, 11, 1), training_acc, label="training_acc")
plt.plot(np.arange(1, 11, 1), test_acc, label="test_acc")
plt.xlabel("n_neighbors")
plt.ylabel("Accuracy")
xticks = np.arange(1, 11, 1)
plt.xticks(xticks)
plt.legend()

In [169]:
y_pred_valid = knn_2c8.predict(x_valid_scaled)

In [170]:
accuracy_score(y_valid, y_pred_valid)

0.8023255813953488

In [171]:
roc_auc_score(y_test, knn_2c8.predict_proba(x_test_scaled)[:,1])

0.832039911308204

In [172]:
matthews_corrcoef(y_test, y_pred)

0.564609859025163

In [173]:
recall_score(y_test, y_pred)

0.7727272727272727

In [174]:
recall_score(y_test, y_pred, pos_label=0)

0.8048780487804879

In [175]:
dump(knn_2c8, '2c8_rdkit_knn.pkl', compress=3)

['2c8_rdkit_knn.pkl']